In [1]:
# -*- coding: utf-8 -*-
"""Esquema de comunicion.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1AOsUFl83ufnTgiWYl9B6Jw23n4J7HHsG
"""

# pip install bitstring

"""# Esquema de comunicación"""

from bitstring import BitArray
import random
import time
import math
import random

#Entropia de shannon
def EntropiaConocida(probabilidad):
  E = -(math.log(probabilidad, 2))
  return E

def EntropiaAleatoria(probabilidades):
  k = len(probabilidades)
  suma = 0
  for i in range(0,k):
    # print(i)
    if probabilidades[i] > 0:
      Ek = probabilidades[i] * math.log(probabilidades[i], 2)
      suma = suma + Ek
  return -suma

#Fuente de información
#pide como entrada un mensaje
def Finformacin():

    nombre = str(input("Escribe tu mensaje "))
    return nombre

#pasa del string a una lista de caracteres
def separar(mensaje):
  lista = []
  for i in mensaje:
    lista.append(i)
  return lista

#convierte el mensaje a binario
def transmite(mensaje):
    # Convertir el mensaje en una secuencia de bits
    bitsM = BitArray(bytes=mensaje.encode('utf-8'))
    return bitsM.bin

#enpaqueta los datos con INIT al inicio y FIN al final de la cadena
def enpaquetar(cadena):
  pack = ["INI"]
  for i in range(len(cadena)):
    pack.append(cadena[i])
  pack.append("FIN")
  return pack

#Reproduce posible ruido sobre las cadenas de bits
#entra: palabra de 8 bits
#sale: palabra con 8 bits con posible cambio en alguno de los bits
def ruido(LetraBytes):
  causa = False
  letra = LetraBytes
  aleatorio1 = random.randint(1, 10)
  # aleatorio2 = random.randint(1, 5)
  if aleatorio1 < 3:
    causa = True
    aleatorioBytes = random.randint(0,7)
    # print(aleatorioBytes,letra[aleatorioBytes])
    if letra[aleatorioBytes] == "0":
      # print("cambio a 1",aleatorioBytes)
      letra = letra[:aleatorioBytes] + '1' + letra[aleatorioBytes + 1:]
    else:
      # print("cambio a 0",aleatorioBytes)
      letra = letra[:aleatorioBytes] + '0' + letra[aleatorioBytes + 1:]


  return letra,causa

#Crea ruido en el mensaje y envia los mensajes cada  0.08 segundos.
def canal(letraB):
  letraR,CausaR = ruido(letraB)
  # print(CausaR)
  # print(letraR)
  velocidad_mbps = 100
  tiempo_teórico = 8 / velocidad_mbps  # Convertimos megabits a bits
  time.sleep(tiempo_teórico)
  verdad = random.randint(0, 1)
  if(verdad):
    print("transferiendo")
  return letraR,CausaR

#Decodifica el mensaje de binario a carácter
def decodifica(letraB):
  #  bits a decimal
  decimal = int(letraB, 2)
  # decimal a carácter
  caracter = chr(decimal)
  return caracter

#Recibe el mensaje enpaquetado y lo decodifica de binario a caracter
def receptor(listaB):
  MensajeFin = []
  listaRuido = []
  # print(len(listaB)-2)
  if((listaB[0] == 'INI') & (listaB[len(listaB)-1] == 'FIN')):
    for i in range(1,len(listaB)-1):
      # print(listaB[i])
      BinR,CausaC = canal(listaB[i])
      #guardar probabilidades
      if(CausaC):
        listaRuido.append(1/(len(listaB)-2))
      else:
        listaRuido.append(0)

      Letra = decodifica(BinR)
      MensajeFin.append(Letra)
    print("lista de probabilidades: ",listaRuido)
    print("Entropia de shannon: ",EntropiaAleatoria(listaRuido))
  else:
    print("EL mensaje no se encuentra enpaquetado")

  return MensajeFin

#recibe la lista de caracteres y las pasa a un string
def Destino(Mensaje):
  MensajeDestino = ''.join(Mensaje)
  print("Mensaje Recibido... :",MensajeDestino)

# #Escribir mensaje
# mensaje = Finformacin()
# print("Mensaje enviado.... :",mensaje)

# #pasar el mensaje a una lista
# ListMensaje = separar(mensaje)

# #convertir cada uno de los caracteres a binario
# BinariList = [transmite(caracter) for caracter in ListMensaje]

# #enpaquetar los datos
# packBList = enpaquetar(BinariList)
# #recibir y decodificar los datos
# entrega = receptor(packBList)

# #recbir los datos finales y convertir de lista a string
# Destino(entrega)

"""# Aplicar Huffman, shannnon y arbol binario

## Aplicar Huffman, shannnon y arbol binario
"""

import copy # para objetos más complejos o personalizados

class Nodo:
    def __init__(self, dato, izquieda=None, derecho=None):
        self.dato = dato
        self.izquieda = izquieda
        self.derecho = derecho

class Arbol:
    # Funciones privadas
    def __init__(self, dato):
        self.raiz = Nodo(dato)

    #AGREGAR DATOS POR HUFFMAN
    def __inicializar_arbol(self,nodo,dato1,dato2):
      #Obtner raiz
      if isinstance(dato1, Nodo):
        # print(1)
        if dato1.dato <= dato2:
          self.__agregar_nodo_izquieda(nodo,dato2) #EL MENOR
          self.__agregar_nodo_derecho(nodo,dato1)
        else:
          self.__agregar_nodo_izquieda(nodo,dato1) #EL MENOR
          self.__agregar_nodo_derecho(nodo,dato2)

      if isinstance(dato2, Nodo):
        # print(2)
        if dato1 <= dato2.dato:
          self.__agregar_nodo_izquieda(nodo,dato2) #EL MENOR
          self.__agregar_nodo_derecho(nodo,dato1)
        else:
          self.__agregar_nodo_izquieda(nodo,dato1) #EL MENOR
          self.__agregar_nodo_derecho(nodo,dato2)

      if (not isinstance(dato1, Nodo)) & (not isinstance(dato1, Nodo)) :
        # print(3)
        if dato1 <= dato2:
          self.__agregar_nodo_izquieda(nodo,dato2) #EL MENOR
          self.__agregar_nodo_derecho(nodo,dato1)
        else:
          self.__agregar_nodo_izquieda(nodo,dato1) #EL MENOR
          self.__agregar_nodo_derecho(nodo,dato2)

    #AGREGAR DATOS POR SHANON FANON
    def __agregar_recursivo(self, nodo): #para shannon
        if len(nodo.dato) == 2:
          # print("lista",nodo.dato, "tamaño", len(nodo.dato))
          nodo.izquieda = Nodo([nodo.dato[0]])
          nodo.derecho = Nodo([nodo.dato[1]])


        elif len(nodo.dato) == 1:
          # print("lista",nodo.dato, "tamaño", len(nodo.dato))
          nodo.izquieda = Nodo([nodo.dato[0]])


        else:
          lista_separar = nodo.dato
          # print("lista completa",lista_separar, "tamaño", len(lista_separar))

          mitad = math.floor(len(lista_separar) /2) #redondear hacia abajo
          mitad1 = lista_separar[0:mitad]
          mitad2 = lista_separar[mitad:len(lista_separar)]
          # print("mitad1",mitad1)
          # print("mitad2",mitad2)
          nodo.izquieda = Nodo(mitad1)
          nodo.derecho = Nodo(mitad2)
          self.__agregar_recursivo(nodo.izquieda)
          self.__agregar_recursivo(nodo.derecho)

    #AGREGAR DATOS POR UN ARBOL BINARIO
    def __agregar_arbol_binario(self, nodo, dato):
        if dato < nodo.dato:
            if nodo.izquieda is None:
                nodo.izquieda = Nodo(dato)
            else:
                self.__agregar_arbol_binario(nodo.izquieda, dato)
        else:
            if nodo.derecho is None:
                nodo.derecho = Nodo(dato)
            else:
                self.__agregar_arbol_binario(nodo.derecho, dato)

    # BUSCAR DATOS POR SHANNON FANON
    def __buscar_shannon(self, nodo, busqueda,palabra=""):

        # print("inicia izquierda", nodo.izquieda.dato)

        if len(nodo.izquieda.dato) == 1:

          if nodo.izquieda.dato[0] == busqueda:
                return "0" + palabra

        # print("inicia derecho", nodo.derecho.dato)
        if len(nodo.derecho.dato) == 1:

          if nodo.derecho.dato[0] == busqueda:
                return "1" + palabra

        if busqueda in nodo.izquieda.dato:
          # print("entra izquierda",nodo.izquieda.dato )
          return self.__buscar_shannon(nodo.izquieda,busqueda,"0" + palabra)
        else:
          # print("entra derecha",nodo.derecho.dato )
          return self.__buscar_shannon(nodo.derecho,busqueda,"1" + palabra)

    #BUSCAR DATOS POR ARBOL BINARIO
    def __buscar_arbol_binario(self, nodo, busqueda,palabra=""):
        if nodo is None:
            return None
        if nodo.dato == busqueda:
            return palabra
        if busqueda < nodo.dato:
            return self.__buscar_arbol_binario(nodo.izquieda, busqueda,"0" + palabra)
        else:
            return self.__buscar_arbol_binario(nodo.derecho, busqueda,"1" + palabra)

    def __agregar_nodo_derecho(self,nodo,dato1):
      if isinstance(dato1, Nodo):
        nodo.derecho = dato1
        # print("nodo_derecho")
      else:
        nodo.derecho = Nodo(dato1) #penultimo menor
        # print("numero_derecho")

    def __agregar_nodo_izquieda(self,nodo,dato1):
      if isinstance(dato1, Nodo):
        nodo.izquieda = dato1
        # print("nodo_izquierdo")
      else:
        nodo.izquieda = Nodo(dato1) #penultimo menor
        # print("numero_izquierdo")

    #BUSCAR DATOS POR HUFFMAN
    def __buscar(self, nodo, busqueda,palabra=""):

        if(nodo.izquieda.dato == busqueda) | (nodo.derecho.dato == busqueda):

          if(nodo.izquieda.dato == busqueda):
            # print(busqueda,"encontrado en izquierda")
            return "0" + palabra
          elif(nodo.derecho.dato == busqueda):
            # print(busqueda,"encontrado en derecho")
            return "1" + palabra
        else:

          if (nodo.derecho.izquieda is not None) | (nodo.derecho.derecho is not None) :

            return self.__buscar(nodo.derecho, busqueda, "1" + palabra)

          else:
            return self.__buscar(nodo.izquieda, busqueda,"0" + palabra)



    def __inorden_recursivo(self, nodo):
        if nodo is not None:
            self.__inorden_recursivo(nodo.izquieda)
            print(nodo.dato, end=", ")
            self.__inorden_recursivo(nodo.derecho)

    def __preorden_recursivo(self, nodo):
        if nodo is not None:
            print(nodo.dato, end=", ")
            # print(nodo.derecho.dato)
            self.__preorden_recursivo(nodo.izquieda)
            self.__preorden_recursivo(nodo.derecho)

    def __postorden_recursivo(self, nodo):
        if nodo is not None:
            self.__postorden_recursivo(nodo.izquieda)
            self.__postorden_recursivo(nodo.derecho)
            print(nodo.dato, end=", ")

    def inorden(self):
        print("Imprimiendo árbol inorden: ")
        self.__inorden_recursivo(self.raiz)
        print("")

    def preorden(self):
        print("Imprimiendo árbol preorden: ")
        self.__preorden_recursivo(self.raiz)
        print("")

    def postorden(self):
        print("Imprimiendo árbol postorden: ")
        self.__postorden_recursivo(self.raiz)
        print("")

    def agregar(self):
        self.__agregar_recursivo(self.raiz)

    def agregar_ini(self, dato1,dato2):
        self.__inicializar_arbol(self.raiz, dato1,dato2)

    def buscar(self, busqueda):
        return self.__buscar(self.raiz, busqueda)

    def buscar_shannon(self, busqueda):
        return self.__buscar_shannon(self.raiz, busqueda)

    def agregar_arbol_binario(self, dato):
        self.__agregar_arbol_binario(self.raiz, dato)

    def buscar_arbol_binario(self, busqueda):
        return self.__buscar_arbol_binario(self.raiz, busqueda)

#Ralizar arbol de huffman
def arbol_huffman(lista_ordenada):
  ultimo = lista_ordenada[len(lista_ordenada)-1]
  penultimo = lista_ordenada[len(lista_ordenada)-2]

  raiz1 = ultimo + penultimo
  Huffman = Arbol(raiz1)
  Huffman.agregar_ini(ultimo,penultimo)
  Huffman_copia = copy.copy(Huffman)

  for i in reversed(range(0, len(lista_ordenada)-2)):

    if i != 0:
      raiz = lista_ordenada[i] + raiz1
      Huffman_principal = Arbol(raiz)
      Huffman_principal.agregar_ini(Huffman_copia.raiz,lista_ordenada[i])
      Huffman_copia = copy.copy(Huffman_principal)
    else:
      Huffman_principal = Arbol(None)
      Huffman_principal.agregar_ini(Huffman_copia.raiz,lista_ordenada[i])

  return Huffman_principal

#aplicar el cifrado RLE a una cadena
def cifrar_RLE(cadena):
  RLE = []
  #incializar contador en 1
  contador = 1
  simbolo = cadena[0]
  #ingresar el primer simbolo
  RLE.append(cadena[0])
  for i in range(1,len(cadena)):
    #si es igual al siguiente elemento incrmenetar contador
    if(simbolo == cadena[i]):
        contador = contador + 1
    else:
      #si es diferente cambiar contador a 1  y el simbolo
      #insertar en la cola el valor del contador y el simbolo
        RLE.append(contador)
        RLE.append(cadena[i])
        contador = 1
        simbolo = cadena[i]

  RLE.append(contador)
  # converitir cada elemento a string
  Elemento_RLE = [str(elemento) for elemento in RLE]

  # unir la cadena en un string
  cadena_RLE = ''.join(Elemento_RLE)
  return cadena_RLE

"""##Esquema de comunicación con huffman y shannnon"""

from collections import Counter

#obtener frecuencias relativas y ordenarlas de mayor a menor
def Obtener_frecuencias_relativas(lista_patrones):

  # Calcular la frecuencia de cada palabra
  contador = Counter(lista_patrones)
  # print(contador)
  # Calcular el número total de palabras en la lista
  total_palabras = len(lista_patrones)

  palabras = []
  frecuencias_relativas = []

  # Calcular la frecuencia relativa de cada palabra y agregarla a las listas
  for palabra, count in contador.items():
      palabras.append(palabra)
      frecuencia_relativa = count / total_palabras
      frecuencias_relativas.append(frecuencia_relativa)
  # Ordenar las palabras y frecuencias juntas en función de las frecuencias (de mayor a menor)
  palabras_ordenadas, frecuencias_ordenadas = zip(*sorted(zip(palabras, frecuencias_relativas), key=lambda x: x[1], reverse=True))

  # Convertir la tupla en una lista
  palabras_ordenadas = list(palabras_ordenadas)
  frecuencias_ordenadas = list(frecuencias_ordenadas)

  #hacer una diferencia para aquellos numero que sean iguales
  for i in range(0,len(frecuencias_ordenadas)-1):
    suma =1e-12
    for j in range(i,len(frecuencias_ordenadas)-1):
      if(frecuencias_ordenadas[i] == frecuencias_ordenadas[j+1]):
        # print(i," ", frecuencias_ordenadas[i]," ",j+1,frecuencias_ordenadas[j+1])
        frecuencias_ordenadas[j+1] = frecuencias_ordenadas[j+1] - suma
        # print(i," ", frecuencias_ordenadas[i]," ",j+1,frecuencias_ordenadas[j+1])
        suma = suma + 1e-12
      else:
        break;

  return palabras_ordenadas,frecuencias_ordenadas

#agarra la lista de bits y codifica a huffman
def Transmisor_huffman(BinariList):
  #obtner frecuencias y patrones ordenados
  patrones,frecuencias = Obtener_frecuencias_relativas(BinariList)
  Huffman = arbol_huffman(frecuencias)

  lista_Huffman = []
  for i in frecuencias:
    lista_Huffman.append(Huffman.buscar(i))
    # print(i," ",Huffman.buscar(i))
  return dict(zip(patrones, lista_Huffman))

#agarra la lista de bits y codifica a huffman
def Transmisor_shannon(BinariList):
  #obtner frecuencias y patrones ordenados
  patrones,frecuencias = Obtener_frecuencias_relativas(BinariList)
  Shannon = Arbol(frecuencias)
  Shannon.agregar()

  lista_Huffman = []
  for i in frecuencias:
    lista_Huffman.append(Shannon.buscar_shannon(i))
    # print(i," ",Huffman.buscar(i))
  return dict(zip(patrones, lista_Huffman))

#agarra la lista de bits y codifica a arbol binario
def Transmisor_arbol_binario(BinariList):
  #obtner frecuencias y patrones ordenados
  patrones,frecuencias = Obtener_frecuencias_relativas(BinariList)

  # Combina las dos listas en una sola lista de tuplas
  combinadas = list(zip(patrones, frecuencias))

  # Baraja la lista combinada
  random.shuffle(combinadas)

  # Divide la lista combinada en las dos listas originales
  patrones_barajadas, frecuencias_barajadas = zip(*combinadas)

  # print(patrones_barajadas)
  # print(frecuencias_barajadas)

  promedio = sum(frecuencias_barajadas) / len(frecuencias_barajadas)

  arbol_binario = Arbol(promedio)
  for i in range(0,len(frecuencias_barajadas)):
    arbol_binario.agregar_arbol_binario(frecuencias_barajadas[i])


  lista_arbol_binario = []
  for i in frecuencias_barajadas:
    lista_arbol_binario.append(arbol_binario.buscar_arbol_binario(i))
    # print(i," ",Huffman.buscar(i))
  return dict(zip(patrones_barajadas, lista_arbol_binario))

def Transmisor_RLE(BinariList):
  #obtener frecuencias y patrones ordenados(solo se usaran los patrones)
  patrones,frecuencias = Obtener_frecuencias_relativas(BinariList)

  lista_RLE = []
  for i in patrones:
    lista_RLE.append(cifrar_RLE(i))

  return dict(zip(patrones,lista_RLE))

def codificar_lista(BinariList,handshake):
  codificar = []
  for i in BinariList:
    codificar.append(handshake[i])
  return codificar

def Decoficar_lista(Lista_codificada,handshake):
  decodificar = []
  #cambia llaves por valores en el diccionario :D
  handshake_reglas_invertido = {valor: clave for clave, valor in handshake_reglas.items()}

  for i in Lista_codificada:
    decodificar.append(handshake_reglas_invertido[i])
  return decodificar

def convertir_caracter(lista_decodificada):
  mensaje = []
  for i in lista_decodificada:
    #  bits a decimal
    decimal = int(i, 2)
    # decimal a carácter
    caracter = chr(decimal)
    mensaje.append(caracter)
  cadena = ''.join(mensaje)
  return cadena

def Destino(Mensaje_binario):
  Mensaje = convertir_caracter(mensajeDecodificado)
  print(Mensaje)

"""# Implementar canales y HASH

## Canal
"""

import random

#elimina un valor aleatorio a la misma posicion de amabs listas
#regresa dos listas y indice del valor eliminado
def eliminar_aleatorio(lista1, lista2):
    if lista1 and lista2:  # Verificar si ambas listas no están vacías
        indice_aleatorio = random.randint(0, min(len(lista1), len(lista2)) - 1)
        lista1.pop(indice_aleatorio)
        valor_eliminado = lista2[indice_aleatorio]
        lista2.pop(indice_aleatorio)
        return lista1,lista2,valor_eliminado

# # Ejemplo de uso:
# mi_lista = [1, 2, 3, 4, 9, 5]
# indices = [11,22,33,34,35]
# lita,indi,indice_eliminado = eliminar_aleatorio(mi_lista,indices)
# print(lita,indi,indice_eliminado)

#actualiza los valores del canal y implementa ruido a los canales
#salida:
# canales modificados
# verdad ruido: para indentificar si entro a un canal
# canal usado : conocer el canal por el cual transmitio los datos
# indice eliminado: conocer el indice del valor eliminado
def canal(paquete,indices,canales):
  # print("mete canal",canales)
  # print(paquete)
  # print(indices)
  #verifica si entro a un canal
  verdad = False
  #verifica si se aplico ruido
  verdad_ruido = False
  canal_usado = None
  indice_eliminado = None
  for i in range(0,len(canales)):
    #revisa si hay ruido
    if(canales[i][0] == 0):
      verdad = True
      canal_usado = i
      # print("entra",i)

      if(random.randint(0, 1)):
        # print("entra random")
        canales[i][0] = 1

      #meter paquete y indices al canal
      canales[i][1] = paquete
      canales[i][2] = indices
      # print("meter",canales)
      if(canales[i][0] == 1):
        # print("mete ruido")
        verdad_ruido = True
        canales[i][1], canales[i][2],indice_eliminado = eliminar_aleatorio(canales[i][1],canales[i][2])
        # print("cambio",canales[i][1], canales[i][2],indice_eliminado)

        ############## RESOLVER COMO BUSCAR EL INDICE FALTANTE
        #si pone ruido en el ultimo canal, quitar ruido en todos los canales
        for j in range(0,len(canales)):
          canales[j][0] = 0


      break

  #Recetear canal
  # if(verdad == False):
  #   for i in range(0,len(canales)):
  #     canales[i][0] = 0

  return canales,verdad_ruido,canal_usado,indice_eliminado

"""## Busqueda binaria y separar diccionario"""

def busqueda_binaria(lista_valores, buscar):
    inicio = 0
    fin = len(lista_valores) - 1
    #si inicio es mayor a fin se termina
    while inicio <= fin:
        #calcular la mitad
        medio = (inicio + fin) // 2

        #si lo  encuentra termina
        if lista_valores[medio] == buscar:
            return medio
        #si la busqueda es menor al valor del medio, el final se marca como medio -1
        if buscar < lista_valores[medio]:
            fin = medio - 1
        #si la busqueda es mayor al valor del medio, se cambia el inicio como medio + 1
        else:
            inicio = medio + 1
    # Si salimos del ciclo significa que no existe el valor
    return -1

#separa el diccionario en claves y valores
#claves y valores se ordenan deacuerdo al sort de valores
#returna clavaes y valores ordenados por el sort de valores
def OrdenarSeparar(diccionario):
  claves = [str(clave) for clave in diccionario.keys()]
  valores = [str(valor) for valor in diccionario.values()]
  # Combina las listas de claves y valores
  claves_y_valores = list(zip(claves, valores))

  # Ordena la lista combinada por los valores
  claves_ordenadas = [clave for clave, valor in sorted(claves_y_valores, key=lambda x: x[1])]
  valores.sort()
  return claves_ordenadas,valores

"""##Función generar Hash y comparar HASH en la ultima parte del receptor"""

#Crear HASH
import hashlib
def generaHash(dato):
  #converitr a binario
  bdatos = bytes(dato,'utf-8')
  #intrucciones
  h = hashlib.new("sha256",bdatos)
  #convertir
  digest=h.hexdigest()
  return digest

#Comparar Hash
#cadenaList - cadena en HASH
#Diccionario - reglas
def CompararHash(cadenaList,claves,valores):
  indice = busqueda_binaria(valores, cadenaList)
  if(indice != -1):
    # print(indice)
    return claves[indice]
  else:
    #Return "?" en binario
    return "00111111"

def Decoficar_lista(Lista_codificada,handshake):
  decodificar = []
  #cambia llaves por valores en el diccionario :D
  # handshake_reglas_invertido = {valor: clave for clave, valor in handshake_reglas.items()}
  #par las claves a Hash
  diccionario_hash = {clave: generaHash(valor) for clave, valor in handshake.items()}
  claves_ordenados,valores_ordenados = OrdenarSeparar(diccionario_hash)

  #Recorrer lista
  for i in Lista_codificada:
    decodificar.append(CompararHash(i,claves_ordenados,valores_ordenados))
  return decodificar

#envia 4 paquetes al canal,recibe el paquete completo y lo decodifica a binario
#devuelve una lista con los valores decodifcados
def Receptor(mensaje_codificado,reglas):
  #inicializar canales
  canales_usado = [[0,None,None],[0,None,None],[0,None,None],[0,None,None]]
  paquete = []
  indices = []
  recibe_paquetes = lista_nula = [None] * len(mensaje_codificado)

  #inicializar tamaño y contador
  tamaño = len(mensaje_codificado)
  contador = 0
  suma = 4

  #envia 4 paquetes al canal
  #termina cuando el contador es mayor a tamaño
  while contador < tamaño:
    for i in range(contador,contador+suma): #aqui
      # print(contador,contador + suma)

      #agarra paquetes de 4
      if i != (tamaño):
        # print(i,mensaje_codificado[i])
        paquete.append(mensaje_codificado[i])
        indices.append(i)
      else:
        contador = contador + suma
        break
    contador = contador + suma
    # print("indice y paquetes",paquete,indices)

    #meter paquetes a canal
    #canales_usado: requesa canales con información
    #indice_eliminado: si hay elementos significa que se le aplico ruido
    #canal usado: conocer el canal que fue usado para transmitir el mensaje

    canales_usado,verdad_ruido,canal_usado,indice_eliminado = canal(paquete,indices,canales_usado)

    paquete = []
    indices = []

    #guardar valores que no se pudieron transmitir
    if indice_eliminado == None:
      suma = 4
    else:
      suma = 3
      # contador = contador - 1
      paquete.append(mensaje_codificado[indice_eliminado])
      indices.append(indice_eliminado)

    #ingresar valores obtenidos en una lista
    valores_obtenido = canales_usado[canal_usado][1]
    indices_de_valores = canales_usado[canal_usado][2]
    # print("paquete obtenido",valores_obtenido,indices_de_valores)

    for valorEnviado, indiceEnviado in zip(valores_obtenido, indices_de_valores):
      recibe_paquetes[indiceEnviado] = valorEnviado

    # print(recibe_paquetes)

  #Revisa si quedaron paquetes en la lista y lo engresa al paquete final
  if paquete:
    # print("no esta vacia",paquete[0],indices[0])
    recibe_paquetes[indices[0]] = paquete[0]

  #-----------------------------------------------------------
  #Al hacer la decoficación igual revisa si la comparación con el Hash es verdadera
  # reglas_hash = [generaHash(cadena) for cadena in reglas]
  decodificar = Decoficar_lista(recibe_paquetes,reglas)
  return decodificar

"""#Esquema de comunicación con canal

**El farsante - Ozuna**

Mi libertad no la quiero Tampoco la vida de soltero Yo lo que quiero es que quieras lo mismo que todos queremos Tener una cuenta de banco con dígitos y muchos ceros Hacer el amor a diario y de paso gastar el dinero
"""

#Escribir mensaje
# mensaje = Finformacin()
# print("Mensaje enviado.... :",mensaje)
mensaje = "Miii libertad no la quiero Tampoco la vida de soltero Yo lo que quiero es que quieras lo mismo que todos queremos Tener una cuenta de banco con digitos y muchos ceros Hacer el amor a diario y de paso gastar el dinero"
# mensaje = "Y baila lento flor morada, que me recuerdas a mi amada Ella me esta esperando en casa y yo muriendo por volver Asi es la vida flor morada, a veces suele ser malvada Tu de mi estas enamorada y yo muriendo por volver muriendo por volver, muriendo por volver"
print("mensaje enviado")
print(mensaje)

#poner opción
#############
verdad = 3  #
#############

#pasar el mensaje a una lista
ListMensaje = separar(mensaje)
#convertir cada uno de los caracteres a binario
BinariList = [transmite(caracter) for caracter in ListMensaje]

#realizar regla de codificación segun el agoritmo
if verdad == 0:
  handshake_reglas = Transmisor_huffman(BinariList)
elif verdad == 1:
  handshake_reglas = Transmisor_shannon(BinariList)
elif verdad == 2:
  handshake_reglas = Transmisor_arbol_binario(BinariList)
elif verdad == 3:
  handshake_reglas = Transmisor_RLE(BinariList)

#agarrar cada paquete y codificarlo
codificar =  codificar_lista(BinariList,handshake_reglas)
#Pasar a Hash
Hash = [generaHash(cadena) for cadena in codificar]

print("\nMensaje codificado")
print(codificar)
print("\nMensaje en SHA256")
print(Hash)
print("\nreglas ")
for clave, valor in handshake_reglas.items():
    print(f"{clave}: {valor}")

#transmir el mensaje por el receptor en SHA256
mensajeDecodificado = Receptor(Hash,handshake_reglas)

#recibir el mensaje
print("\nmensaje")
Destino(mensajeDecodificado)

mensaje enviado
Miii libertad no la quiero Tampoco la vida de soltero Yo lo que quiero es que quieras lo mismo que todos queremos Tener una cuenta de banco con digitos y muchos ceros Hacer el amor a diario y de paso gastar el dinero

Mensaje codificado
['011102120111', '011201110211', '011201110211', '011201110211', '021105', '0112011202', '011201110211', '0112031101', '011202110111', '0113021101', '0113011102', '01120411', '0112021102', '021105', '0112011301', '01120114', '021105', '0112011202', '01120411', '021105', '01130311', '011301110111', '011201110211', '011202110111', '0113021101', '01120114', '021105', '01110111011102', '01120411', '011201120111', '011304', '01120114', '01120312', '01120114', '021105', '0112011202', '01120411', '021105', '0113011201', '011201110211', '0112021102', '01120411', '021105', '0112021102', '011202110111', '021105', '01130212', '01120114', '0112011202', '0113011102', '011202110111', '0113021101', '01120114', '021105', '011101120211', '01120114', '021